In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pyspark-logistic-regression/Log_Reg_dataset.csv


In [2]:
!pip install findspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 KB 15.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=de5ddda0ae16ca0211181484a33bbbad2aebd687db12e4aa968cc1d4b717ba50
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.5
    Uninstalling py4j-0.10.9.5:
      Successfully uninstalled py4j-0.10.9.5


In [3]:
import findspark
findspark.init()
import pyspark

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('log_reg').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/15 17:48:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
#import libs
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.classification import LogisticRegression

In [6]:
#read the dataset
df = spark.read.csv('/kaggle/input/pyspark-logistic-regression/Log_Reg_dataset.csv', inferSchema=True,header=True)
df.show(5)

+---------+---+--------------+--------+----------------+------+
|  Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|
+---------+---+--------------+--------+----------------+------+
|    India| 41|             1|   Yahoo|              21|     1|
|   Brazil| 28|             1|   Yahoo|               5|     0|
|   Brazil| 40|             0|  Google|               3|     0|
|Indonesia| 31|             1|    Bing|              15|     1|
| Malaysia| 32|             0|  Google|              15|     1|
+---------+---+--------------+--------+----------------+------+
only showing top 5 rows



In [7]:
#shape of data
print((df.count(),len(df.columns)))

(20000, 6)


In [8]:
#check the schemea
df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Repeat_Visitor: integer (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Web_pages_viewed: integer (nullable = true)
 |-- Status: integer (nullable = true)



In [9]:
#explore the dataset
df.describe().show()

+-------+--------+-----------------+-----------------+--------+-----------------+------------------+
|summary| Country|              Age|   Repeat_Visitor|Platform| Web_pages_viewed|            Status|
+-------+--------+-----------------+-----------------+--------+-----------------+------------------+
|  count|   20000|            20000|            20000|   20000|            20000|             20000|
|   mean|    null|         28.53955|           0.5029|    null|           9.5533|               0.5|
| stddev|    null|7.888912950773227|0.500004090187782|    null|6.073903499824976|0.5000125004687693|
|    min|  Brazil|               17|                0|    Bing|                1|                 0|
|    max|Malaysia|              111|                1|   Yahoo|               29|                 1|
+-------+--------+-----------------+-----------------+--------+-----------------+------------------+



In [10]:
df.groupby('country').count().show()

+---------+-----+
|  country|count|
+---------+-----+
| Malaysia| 1218|
|    India| 4018|
|Indonesia|12178|
|   Brazil| 2586|
+---------+-----+



In [11]:
df.groupby('Platform').count().show()

+--------+-----+
|Platform|count|
+--------+-----+
|   Yahoo| 9859|
|    Bing| 4360|
|  Google| 5781|
+--------+-----+



In [12]:
df.groupby('Status').count().show()

+------+-----+
|Status|count|
+------+-----+
|     1|10000|
|     0|10000|
+------+-----+



In [13]:
df.groupby('Country').mean().show()

+---------+------------------+-------------------+---------------------+--------------------+
|  Country|          avg(Age)|avg(Repeat_Visitor)|avg(Web_pages_viewed)|         avg(Status)|
+---------+------------------+-------------------+---------------------+--------------------+
| Malaysia|27.792282430213465| 0.5730706075533661|   11.192118226600986|  0.6568144499178982|
|    India|27.976854156296664| 0.5433051269288203|   10.727227476356397|  0.6212045793927327|
|Indonesia| 28.43159796354081| 0.5207751683363442|    9.985711939563148|  0.5422893742814913|
|   Brazil|30.274168600154677|  0.322892498066512|    4.921113689095128|0.038669760247486466|
+---------+------------------+-------------------+---------------------+--------------------+



In [14]:
df.groupby('Platform').mean().show()

+--------+------------------+-------------------+---------------------+------------------+
|Platform|          avg(Age)|avg(Repeat_Visitor)|avg(Web_pages_viewed)|       avg(Status)|
+--------+------------------+-------------------+---------------------+------------------+
|   Yahoo|28.569226087838523| 0.5094837204584644|    9.599655137437875|0.5071508266558474|
|    Bing| 28.68394495412844| 0.4720183486238532|    9.114908256880733|0.4559633027522936|
|  Google|28.380038055699707| 0.5149628092025601|    9.804878048780488|0.5210171250648676|
+--------+------------------+-------------------+---------------------+------------------+



_Data Preprocessing, converting categorical data into Numerical form_

In [15]:
search_engine_indexer = StringIndexer(inputCol='Platform', outputCol='Search_Engine_Number').fit(df)
df = search_engine_indexer.transform(df)
df.show(3, False)

+-------+---+--------------+--------+----------------+------+--------------------+
|Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|Search_Engine_Number|
+-------+---+--------------+--------+----------------+------+--------------------+
|India  |41 |1             |Yahoo   |21              |1     |0.0                 |
|Brazil |28 |1             |Yahoo   |5               |0     |0.0                 |
|Brazil |40 |0             |Google  |3               |0     |1.0                 |
+-------+---+--------------+--------+----------------+------+--------------------+
only showing top 3 rows



In [16]:
#one hot encoding for Search Engine /Platform Variable
search_engine_encoder = OneHotEncoder(inputCol="Search_Engine_Number", outputCol="Search_Engine_Vector").fit(df)
df = search_engine_encoder.transform(df)
df.show(3,False)

+-------+---+--------------+--------+----------------+------+--------------------+--------------------+
|Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|Search_Engine_Number|Search_Engine_Vector|
+-------+---+--------------+--------+----------------+------+--------------------+--------------------+
|India  |41 |1             |Yahoo   |21              |1     |0.0                 |(2,[0],[1.0])       |
|Brazil |28 |1             |Yahoo   |5               |0     |0.0                 |(2,[0],[1.0])       |
|Brazil |40 |0             |Google  |3               |0     |1.0                 |(2,[1],[1.0])       |
+-------+---+--------------+--------+----------------+------+--------------------+--------------------+
only showing top 3 rows



In [17]:
df.groupby('Platform').count().orderBy('count', ascending=False).show()

+--------+-----+
|Platform|count|
+--------+-----+
|   Yahoo| 9859|
|  Google| 5781|
|    Bing| 4360|
+--------+-----+



In [18]:
df.groupby('Search_Engine_Number').count().orderBy('count', ascending=False).show()

+--------------------+-----+
|Search_Engine_Number|count|
+--------------------+-----+
|                 0.0| 9859|
|                 1.0| 5781|
|                 2.0| 4360|
+--------------------+-----+



In [19]:
df.groupby('Search_Engine_Vector').count().orderBy('count', ascending=False).show()

+--------------------+-----+
|Search_Engine_Vector|count|
+--------------------+-----+
|       (2,[0],[1.0])| 9859|
|       (2,[1],[1.0])| 5781|
|           (2,[],[])| 4360|
+--------------------+-----+



In [20]:
#Applying tranformation for country
#string Indexer
country_indexer = StringIndexer(inputCol='Country', outputCol='Country_Num').fit(df)
df = country_indexer.transform(df)

#OnehotEncoding
country_encoder = OneHotEncoder(inputCol='Country_Num', outputCol='Country_Vector').fit(df)
df = country_encoder.transform(df)

df.show(3, False)

+-------+---+--------------+--------+----------------+------+--------------------+--------------------+-----------+--------------+
|Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|Search_Engine_Number|Search_Engine_Vector|Country_Num|Country_Vector|
+-------+---+--------------+--------+----------------+------+--------------------+--------------------+-----------+--------------+
|India  |41 |1             |Yahoo   |21              |1     |0.0                 |(2,[0],[1.0])       |1.0        |(3,[1],[1.0]) |
|Brazil |28 |1             |Yahoo   |5               |0     |0.0                 |(2,[0],[1.0])       |2.0        |(3,[2],[1.0]) |
|Brazil |40 |0             |Google  |3               |0     |1.0                 |(2,[1],[1.0])       |2.0        |(3,[2],[1.0]) |
+-------+---+--------------+--------+----------------+------+--------------------+--------------------+-----------+--------------+
only showing top 3 rows



In [21]:
df.select(['country', 'Country_Num', 'country_Vector']).show(3,False)

+-------+-----------+--------------+
|country|Country_Num|country_Vector|
+-------+-----------+--------------+
|India  |1.0        |(3,[1],[1.0]) |
|Brazil |2.0        |(3,[2],[1.0]) |
|Brazil |2.0        |(3,[2],[1.0]) |
+-------+-----------+--------------+
only showing top 3 rows



In [22]:
df.groupby('Country').count().orderBy('count', ascending=False).show()

+---------+-----+
|  Country|count|
+---------+-----+
|Indonesia|12178|
|    India| 4018|
|   Brazil| 2586|
| Malaysia| 1218|
+---------+-----+



In [23]:
df.groupby('Country_Num').count().orderBy('count', ascending=False).show()

+-----------+-----+
|Country_Num|count|
+-----------+-----+
|        0.0|12178|
|        1.0| 4018|
|        2.0| 2586|
|        3.0| 1218|
+-----------+-----+



In [24]:
df.groupby('Country_Vector').count().orderBy('count', ascending=False).show()

+--------------+-----+
|Country_Vector|count|
+--------------+-----+
| (3,[0],[1.0])|12178|
| (3,[1],[1.0])| 4018|
| (3,[2],[1.0])| 2586|
|     (3,[],[])| 1218|
+--------------+-----+



In [25]:
df.columns

['Country',
 'Age',
 'Repeat_Visitor',
 'Platform',
 'Web_pages_viewed',
 'Status',
 'Search_Engine_Number',
 'Search_Engine_Vector',
 'Country_Num',
 'Country_Vector']

In [26]:
df_assembler = VectorAssembler(inputCols=[
#'Country',
 'Age',
 'Repeat_Visitor',
 #'Platform',
 'Web_pages_viewed',
 'Status',
# 'Search_Engine_Number',
 'Search_Engine_Vector',
 #'Country_Num',
 'Country_Vector'], outputCol='features')
df = df_assembler.transform(df)

In [27]:
df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Repeat_Visitor: integer (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Web_pages_viewed: integer (nullable = true)
 |-- Status: integer (nullable = true)
 |-- Search_Engine_Number: double (nullable = false)
 |-- Search_Engine_Vector: vector (nullable = true)
 |-- Country_Num: double (nullable = false)
 |-- Country_Vector: vector (nullable = true)
 |-- features: vector (nullable = true)



In [28]:
df.select(['features', 'Status']).show(10,False)

+---------------------------------------+------+
|features                               |Status|
+---------------------------------------+------+
|[41.0,1.0,21.0,1.0,1.0,0.0,0.0,1.0,0.0]|1     |
|[28.0,1.0,5.0,0.0,1.0,0.0,0.0,0.0,1.0] |0     |
|(9,[0,2,5,8],[40.0,3.0,1.0,1.0])       |0     |
|[31.0,1.0,15.0,1.0,0.0,0.0,1.0,0.0,0.0]|1     |
|(9,[0,2,3,5],[32.0,15.0,1.0,1.0])      |1     |
|(9,[0,2,5,8],[32.0,3.0,1.0,1.0])       |0     |
|(9,[0,2,5,8],[32.0,6.0,1.0,1.0])       |0     |
|(9,[0,2,5,6],[27.0,9.0,1.0,1.0])       |0     |
|(9,[0,2,4,6],[32.0,2.0,1.0,1.0])       |0     |
|[31.0,1.0,16.0,1.0,0.0,0.0,1.0,0.0,0.0]|1     |
+---------------------------------------+------+
only showing top 10 rows



In [29]:
model_df = df.select(['features', 'Status'])
model_df.show(3,False)

+---------------------------------------+------+
|features                               |Status|
+---------------------------------------+------+
|[41.0,1.0,21.0,1.0,1.0,0.0,0.0,1.0,0.0]|1     |
|[28.0,1.0,5.0,0.0,1.0,0.0,0.0,0.0,1.0] |0     |
|(9,[0,2,5,8],[40.0,3.0,1.0,1.0])       |0     |
+---------------------------------------+------+
only showing top 3 rows



In [30]:
#split the data
traindf, testdf = model_df.randomSplit([0.75,0.25], seed=1)

In [31]:
traindf.count()

15023

In [32]:
traindf.groupby('Status').count().show()

+------+-----+
|Status|count|
+------+-----+
|     1| 7476|
|     0| 7547|
+------+-----+



In [33]:
testdf.groupby('Status').count().show()

+------+-----+
|Status|count|
+------+-----+
|     1| 2524|
|     0| 2453|
+------+-----+

